In [ ]:
# baseline cnn model specially designed for 28x28 image training
import numpy as np
from numpy import mean
from numpy import std
from matplotlib import pyplot
from sklearn.model_selection import KFold
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from sklearn.model_selection import train_test_split


# load train and test dataset

def load_dataset():
	# load dataset
	match_train = np.load("match_train.npy", allow_pickle=True)
	spoken_train = np.load("spoken_train.npy", allow_pickle=True)
	written_train = np.load("written_train.npy", allow_pickle=True)
	written_test = np.load("written_test.npy", allow_pickle=True)
	spoken_test = np.load("written_test.npy", allow_pickle=True)
	
	print(written_train.shape)
	print(spoken_train.shape)


    
	# convert images from integers to floats
	written_train_norm = written_train.astype('float32')
	# normalize to range 0-1
	written_train_norm = written_train_norm / 255.0
	# return normalized images
    
	## sample van trainingdata om klein te beginnen, staat nu op de volledige trainingsset
	sample_w = written_train_norm[:, :]
	sample_s = spoken_train[:]
    
	# eerst een lege array aanmaken
	mean_s = np.empty(len(sample_s))
	## mean van spoken wordt berekent in een for loop hieronder, omdat elke sample varieert in N * 13 features
	# for loop om gemiddelde uit te rekenen
	for i in range(0, len(sample_s)):
		mean_s[i] = np.mean(sample_s[i])
	## mean berekenen van Written
	mean_w = np.mean(sample_w, axis=1)
    
	print(mean_s.shape)
	written_spoken_train = np.array([mean_s, mean_w])
	written_spoken_train = written_spoken_train.T
	print("written + spoken training shape:", written_spoken_train.shape)

	#-------------> Test preparation
	sample_w_test = written_test[:, :]
	sample_s_test = spoken_test[:]

	# eerst een lege array aanmaken TEST
	mean_s_test = np.empty(len(sample_s_test))

	## mean van spoken wordt berekent in een for loop hieronder, omdat elke sample varieert in N * 13 features
	# for loop om gemiddelde uit te rekenen
	for i in range(0, len(sample_s_test)):
		mean_s_test[i] = np.mean(sample_s_test[i])

	## mean berekenen van Written TEST
	mean_w_test = np.mean(sample_w_test, axis=1)

	written_spoken_test = np.array([mean_s_test, mean_w_test])
	written_spoken_test = written_spoken_test.T
	print("written + spoken TEST shape:", written_spoken_test.shape)
	########### Preprocessing afgerond

	#inladen in model en data split maken 
	trainX, testX, trainY, testY = train_test_split(written_spoken_train,
                                                  match_train,
                                                  test_size=1 / 3,
                                                  random_state=555)


	# reshape dataset of pictures to have a single color channel because it's in grayscale
	trainX = trainX.reshape((trainX.shape[0], 28, 28, 1))
	testX = testX.reshape((testX.shape[0], 28, 28, 1))


	# one hot encode target values
	trainY = to_categorical(trainY)
	testY = to_categorical(testY)

	return trainX, trainY, testX, testY


# define cnn model
def define_model():
	model = Sequential()
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)))
	model.add(MaxPooling2D((2, 2)))
	model.add(Flatten())
	model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
	model.add(Dense(10, activation='softmax'))
	# compile model
	opt = SGD(lr=0.01, momentum=0.9)
	model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
	return model

# evaluate a model using k-fold cross-validation
def evaluate_model(model, dataX, dataY, n_folds=5):
	scores, histories = list(), list()
	# prepare cross validation
	kfold = KFold(n_folds, shuffle=True, random_state=1)
	# enumerate splits
	for train_ix, test_ix in kfold.split(dataX):
		# select rows for train and test
		trainX, trainY, testX, testY = dataX[train_ix], dataY[train_ix], dataX[test_ix], dataY[test_ix]
		# fit model
		history = model.fit(trainX, trainY, epochs=3, batch_size=32, validation_data=(testX, testY), verbose=0)
		# evaluate model
		_, acc = model.evaluate(testX, testY, verbose=0)
		print('> %.3f' % (acc * 100.0))
		# stores scores
		scores.append(acc)
		histories.append(history)
	return scores, histories

# plot diagnostic learning curves
def summarize_diagnostics(histories):
	for i in range(len(histories)):
		# plot loss
		pyplot.subplot(211)
		pyplot.title('Cross Entropy Loss')
		pyplot.plot(histories[i].history['loss'], color='blue', label='train')
		pyplot.plot(histories[i].history['val_loss'], color='orange', label='test')
		# plot accuracy
		pyplot.subplot(212)
		pyplot.title('Classification Accuracy')
		pyplot.plot(histories[i].history['acc'], color='blue', label='train')
		pyplot.plot(histories[i].history['val_acc'], color='orange', label='test')
	pyplot.show()

# summarize model performance
def summarize_performance(scores):
	# print summary
	print('Accuracy: mean=%.3f std=%.3f, n=%d' % (mean(scores)*100, std(scores)*100, len(scores)))
	# box and whisker plots of results
	pyplot.boxplot(scores)
	pyplot.show()

# run the test harness for evaluating a model
def run_test_harness():
	# load dataset & normalize pixel data & split data for testing
	trainX, trainY, testX, testY = load_dataset()
	# prepare pixel data
	trainX, testX = prep_pixels(trainX, testX)
	# define model
	model = define_model()
	# evaluate model
	scores, histories = evaluate_model(model, trainX, trainY)
	# learning curves
	summarize_diagnostics(histories)
	# summarize estimated performance
	summarize_performance(scores)

# entry point, run the test harness
run_test_harness()